<div align="right">Python 2.7</div>

# Indexing and Related Experiments in Python 2.7
Though this content is in Python 2.7, most if not all of it should work the same in Python 3.x.

## TOC
- [Indexing Experiments](#here) - Explores different complex structures and how to index into them
- [Mutation sidebar](#mutation) - looks at mutation using our crazyList example
- [Finding the Index of a Known Value within Complex Data Structures](#indexing) - explores `.index()`, `np.where()`, and Related concerns

## Indexing Experiments in Python

We start with a simple nested list showing how to get at an element within it:

In [34]:
stupidList = [[1,2,3],[4,5,6]]
print(stupidList)
stupidList[0][1]

[[1, 2, 3], [4, 5, 6]]


2

Now we build something more complicated to show where indexing can get tricky ...

In [35]:
import numpy as np
import pandas as pd

In [36]:
m3d=np.random.rand(3,4,5)
m3d

array([[[ 0.02764211,  0.26386474,  0.88865395,  0.34781755,  0.72534514],
        [ 0.66574007,  0.73384079,  0.57345696,  0.55389339,  0.87147245],
        [ 0.98128298,  0.88944205,  0.44554892,  0.7894838 ,  0.97115509],
        [ 0.08401461,  0.16542205,  0.00558565,  0.86419018,  0.40254914]],

       [[ 0.71687244,  0.72853557,  0.55106175,  0.78393111,  0.69921926],
        [ 0.12197909,  0.93129564,  0.42840254,  0.34314161,  0.12802696],
        [ 0.88812661,  0.75876102,  0.00127025,  0.60469319,  0.28094787],
        [ 0.61414269,  0.69289326,  0.78397186,  0.19069658,  0.76636118]],

       [[ 0.28322395,  0.20178467,  0.69215878,  0.80796972,  0.08010599],
        [ 0.47625811,  0.5169806 ,  0.98614882,  0.26718482,  0.4316306 ],
        [ 0.15617731,  0.80081827,  0.37400983,  0.04256393,  0.98771807],
        [ 0.99770397,  0.42965486,  0.2190626 ,  0.24189908,  0.95881218]]])

In [37]:
# how does Pandas arrange the data?
n3d=m3d.reshape(4,3,5)
n3d

array([[[ 0.02764211,  0.26386474,  0.88865395,  0.34781755,  0.72534514],
        [ 0.66574007,  0.73384079,  0.57345696,  0.55389339,  0.87147245],
        [ 0.98128298,  0.88944205,  0.44554892,  0.7894838 ,  0.97115509]],

       [[ 0.08401461,  0.16542205,  0.00558565,  0.86419018,  0.40254914],
        [ 0.71687244,  0.72853557,  0.55106175,  0.78393111,  0.69921926],
        [ 0.12197909,  0.93129564,  0.42840254,  0.34314161,  0.12802696]],

       [[ 0.88812661,  0.75876102,  0.00127025,  0.60469319,  0.28094787],
        [ 0.61414269,  0.69289326,  0.78397186,  0.19069658,  0.76636118],
        [ 0.28322395,  0.20178467,  0.69215878,  0.80796972,  0.08010599]],

       [[ 0.47625811,  0.5169806 ,  0.98614882,  0.26718482,  0.4316306 ],
        [ 0.15617731,  0.80081827,  0.37400983,  0.04256393,  0.98771807],
        [ 0.99770397,  0.42965486,  0.2190626 ,  0.24189908,  0.95881218]]])

Notice which numbers moved where.  This would seem to indicate that in shape(a,b,c):
- a is like the object's depth (how many groupings of rows/columns are there?)
- b is like the object's rows per grouping (how many rows in each subgroup)
- c is like the object's columns

What if the object had 4 dimensions?

In [38]:
o3d=np.random.rand(2,3,4,5)
o3d

array([[[[ 0.85299096,  0.62408962,  0.46559312,  0.59977211,  0.38041822],
         [ 0.45831929,  0.90751208,  0.55776963,  0.80488569,  0.70041158],
         [ 0.24332968,  0.21271898,  0.44423011,  0.9098522 ,  0.5045675 ],
         [ 0.6565031 ,  0.19822212,  0.39151489,  0.48465936,  0.71523129]],

        [[ 0.96501237,  0.66875908,  0.99435007,  0.51638045,  0.92869465],
         [ 0.76617542,  0.50846374,  0.05004649,  0.79113062,  0.82639575],
         [ 0.20097392,  0.23616426,  0.33070455,  0.71779095,  0.89065204],
         [ 0.24929711,  0.60038916,  0.48634717,  0.35480637,  0.34372284]],

        [[ 0.97387369,  0.71569147,  0.85060481,  0.9277284 ,  0.14078441],
         [ 0.56117986,  0.28928248,  0.95277008,  0.72973045,  0.77361024],
         [ 0.18379655,  0.12094671,  0.67305481,  0.24605122,  0.3468539 ],
         [ 0.65142807,  0.47173961,  0.92538947,  0.41694106,  0.51021661]]],


       [[[ 0.76562398,  0.76076129,  0.72521928,  0.56537478,  0.10063146],
    

Just analyzing how the numbers are arranged, we see that in shape(a,b,c,d), it just added the new extra dimensional layer to the front of the list so that now:
- a = larger hyper grouping (2 of them)
- b = first subgroup within (3 of them)
- c = rows within these groupings (4 of them)
- d = columns within these groupings (5 of them)

It appears that rows always come before columns, and then it looks like groupings of rows and columns and groupings or groupings, etc. . . are added to the front of the index chain.

Building something complex just to drill in more on how to access sub-elements:

In [39]:
# some simple arrays:
simp1=np.array([[1,2,3,4,5]])
simp2=np.array([[10,9,8,7,6]])
simp3=[11,12,13]

In [40]:
# a dictionary
dfrm1 = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002],
        'population': [1.5, 1.7, 3.6, 2.4, 2.9]}
# convert dictionary to DataFrame
dfrm1 = pd.DataFrame(dfrm1)
dfrm1

,population,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002


In [41]:
# pandas indexing works a little differently:
#   * column headers are keys
#   * as shown here, can ask for columns, rows, and a filter based on values in the columns 
#     in any order and the indexing will still work

print(dfrm1["population"][dfrm1["population"] > 1.5][2:4])  # all of these return values from "population" column only
print("---")                                                # where "population" > 1.5
print(dfrm1["population"][2:4][dfrm1["population"] > 1.5])  # and row index is between 2 and 4
print("---")
print(dfrm1[dfrm1["population"] > 1.5]["population"][2:4])
print("---")
print(dfrm1[dfrm1["population"] > 1.5][2:4]["population"])
print("---")
print(dfrm1[2:4]["population"][dfrm1["population"] > 1.5])
print("---")
print(dfrm1[2:4][dfrm1["population"] > 1.5]["population"]) # this last one triggers a warning

3    2.4
4    2.9
Name: population, dtype: float64
---
2    3.6
3    2.4
Name: population, dtype: float64
---
3    2.4
4    2.9
Name: population, dtype: float64
---
3    2.4
4    2.9
Name: population, dtype: float64
---
2    3.6
3    2.4
Name: population, dtype: float64
---
2    3.6
3    2.4
Name: population, dtype: float64


C:\ProgramFilesCoders\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [42]:
# breaking the above apart:
print(dfrm1[dfrm1["population"] > 1.5])  # all rows and columns filtered by "population" values > 1.5
print("---")
print(dfrm1["population"])               # return whole "population" column
print("---")
print(dfrm1[2:4])                        # return whole rows 2 to 4

   population   state  year
1         1.7    Ohio  2001
2         3.6    Ohio  2002
3         2.4  Nevada  2001
4         2.9  Nevada  2002
---
0    1.5
1    1.7
2    3.6
3    2.4
4    2.9
Name: population, dtype: float64
---
   population   state  year
2         3.6    Ohio  2002
3         2.4  Nevada  2001


In [43]:
crazyList = [simp1, m3d, simp2, n3d, simp3, dfrm1, o3d]

In [44]:
# Accessing the dataframe inside the list now that it is a sub element:
crazyList[5]["population"][crazyList[5]["population"] > 1.5][2:4]

3    2.4
4    2.9
Name: population, dtype: float64

Now let's access other stuff in the list ...

In [45]:
crazyList[1]  # this is the second object of the list (Python like many languages starts indicies at 0)
              # this is the full output of m3d

array([[[ 0.02764211,  0.26386474,  0.88865395,  0.34781755,  0.72534514],
        [ 0.66574007,  0.73384079,  0.57345696,  0.55389339,  0.87147245],
        [ 0.98128298,  0.88944205,  0.44554892,  0.7894838 ,  0.97115509],
        [ 0.08401461,  0.16542205,  0.00558565,  0.86419018,  0.40254914]],

       [[ 0.71687244,  0.72853557,  0.55106175,  0.78393111,  0.69921926],
        [ 0.12197909,  0.93129564,  0.42840254,  0.34314161,  0.12802696],
        [ 0.88812661,  0.75876102,  0.00127025,  0.60469319,  0.28094787],
        [ 0.61414269,  0.69289326,  0.78397186,  0.19069658,  0.76636118]],

       [[ 0.28322395,  0.20178467,  0.69215878,  0.80796972,  0.08010599],
        [ 0.47625811,  0.5169806 ,  0.98614882,  0.26718482,  0.4316306 ],
        [ 0.15617731,  0.80081827,  0.37400983,  0.04256393,  0.98771807],
        [ 0.99770397,  0.42965486,  0.2190626 ,  0.24189908,  0.95881218]]])

In [46]:
crazyList[0]  # after the above demo, no surprises here ... simp1 was the first object we added to the list

array([[1, 2, 3, 4, 5]])

In the tests that follow ... anything that does not work is wrapped in exception handling (that displays the error) so this notebook can be run from start to finish ... Note that it is not good practice to use a catch all for all errors. In real coding errors should be handled individually by type.

How do we access the first index (element 2) of the first array object in our complex list (which resides at index 0)?

In [50]:
try:                  # not this way ...
    crazyList[0][1]
except Exception as ex:
    print("%s%s %s" %(type(ex), ":", ex))

<type 'exceptions.IndexError'>: index 1 is out of bounds for axis 0 with size 1


In [12]:
# let's look at what we built:  all the objects are here but are no longer named so we need to get indices right
crazyList

[array([[1, 2, 3, 4, 5]]),
 array([[[ 0.33133919,  0.43768372,  0.34821673,  0.63767749,  0.29417603],
         [ 0.10204308,  0.29877032,  0.46202304,  0.86789637,  0.94000811],
         [ 0.98822145,  0.09783841,  0.07314108,  0.54074233,  0.3720003 ],
         [ 0.74086424,  0.94474234,  0.04963839,  0.48506306,  0.06371589]],
 
        [[ 0.39509039,  0.33289654,  0.32496611,  0.16405267,  0.33689606],
         [ 0.98477636,  0.22949092,  0.95920822,  0.46390272,  0.48553585],
         [ 0.36850929,  0.63691697,  0.09651275,  0.55299958,  0.88034476],
         [ 0.30275465,  0.03114669,  0.76291335,  0.89423036,  0.10291813]],
 
        [[ 0.61434208,  0.23482192,  0.76548015,  0.46095372,  0.15683438],
         [ 0.23001074,  0.75814235,  0.77721088,  0.32257422,  0.30340721],
         [ 0.34649924,  0.62274606,  0.75516247,  0.33786868,  0.7831538 ],
         [ 0.59216801,  0.69653358,  0.31194369,  0.71861906,  0.2152945 ]]]),
 array([[10,  9,  8,  7,  6]]),
 array([[[ 0.3313391

In [51]:
# note that both of these get the same data, but also note the difference in the format: "[[]]" and array([])".
# look at the source and you will see we are drilling in at different levels of "[]"
# there can be situations in real coding where extra layers are created by accident so this example is good to know

print(crazyList[0])
crazyList[0][0]

[[1 2 3 4 5]]


array([1, 2, 3, 4, 5])

Sub element 4 is a simple list nested within caryList: crazyList [ ... [content at index position 4] ...]

In [52]:
print(crazyList[4])
crazyList[4][1]  # get 2nd element in the list within a list at position 4 (object 4 in the list)

[11, 12, 13]


12

So what about the array?  The array was originally built in "simp1" and then added to crazyList.  Its source looks like this:

In [89]:
print(type(simp1))
print(simp1.shape)

<type 'numpy.ndarray'>
(1L, 5L)


In [54]:
print(simp1)
print(simp1[0])  # note that the first two give us the same thing (whole array)
simp1[0][1]

[[1 2 3 4 5]]
[1 2 3 4 5]


2

Note the [] versus the [[]] ... our "simple arrays" were copied from an example, but are actually nested objects of 1 list of 5 elements forming the first object inside the array.  A true simple array would like this:

In [55]:
trueSimp1=np.array([10,9,8,7,6])
print(trueSimp1.shape)             # note:  output shows that Python thinks this is 5 rows, 1 column
trueSimp1

(5L,)


array([10,  9,  8,  7,  6])

Let's add the true simple array to our crazy object and then create working examples of accessing everything ...

In [57]:
crazyList.append(trueSimp1)   # append mutates so this changes the original list
crazyList                     # Warning! if you re-run this cell, you will keep adding more copies of the last object
                              # to the end of this object.  To be consistent with content in this NB
                              # clear and re-run the whole notebook should that happen

[array([[1, 2, 3, 4, 5]]),
 array([[[ 0.02764211,  0.26386474,  0.88865395,  0.34781755,  0.72534514],
         [ 0.66574007,  0.73384079,  0.57345696,  0.55389339,  0.87147245],
         [ 0.98128298,  0.88944205,  0.44554892,  0.7894838 ,  0.97115509],
         [ 0.08401461,  0.16542205,  0.00558565,  0.86419018,  0.40254914]],
 
        [[ 0.71687244,  0.72853557,  0.55106175,  0.78393111,  0.69921926],
         [ 0.12197909,  0.93129564,  0.42840254,  0.34314161,  0.12802696],
         [ 0.88812661,  0.75876102,  0.00127025,  0.60469319,  0.28094787],
         [ 0.61414269,  0.69289326,  0.78397186,  0.19069658,  0.76636118]],
 
        [[ 0.28322395,  0.20178467,  0.69215878,  0.80796972,  0.08010599],
         [ 0.47625811,  0.5169806 ,  0.98614882,  0.26718482,  0.4316306 ],
         [ 0.15617731,  0.80081827,  0.37400983,  0.04256393,  0.98771807],
         [ 0.99770397,  0.42965486,  0.2190626 ,  0.24189908,  0.95881218]]]),
 array([[10,  9,  8,  7,  6]]),
 array([[[ 0.0276421

In [60]:
# The elements at either end of crazyList:
print(crazyList[0])
print(crazyList[-1])  # ask for last item by counting backwards from the end

[[1 2 3 4 5]]
[10  9  8  7  6]


In [63]:
# get a specific value by index from within the subelements at either end:
print(crazyList[0][0][2]) # extra zero for the extra [] .. structurally this is really [0 [0 ], [1] ] but 1 does not exist
print(crazyList[-1][2])

3
8


Looking at just that first element again:

In [40]:
crazyList[0]  # first array to change

array([[1, 2, 3, 4, 5]])

remember that this object if it were not in a list would be accessed like so:

In [41]:
simp1[0][1]     # second element inside it

2

... so inside crazyList ?  The answer is that the list is one level deep and the elements are yet another level in:

In [115]:
crazyList[0]

array([[1, 2, 3, 4, 5]])

In [116]:
crazyList[0][0][1]

2

<a id="mutation" name="mutation"></a>
## Sidebar:  Mutation and Related Concerns

Try this test and you will see it does not work:

crazyList2 = crazyList.append(trueSimp1)

What it did:  crazyList got an element appended to the end and crazyList2 came out the other side empty.  This is because append() returns None and operates on the original. The copy then gets nothing and the original gets an element added to it.

To set up crazyList2 to append to only it, we might be tempted to try something like what is shown below, but if we do, note how it mutates:

In [64]:
aList = [1,2,3]
bList = aList
print(aList)
print(bList)

[1, 2, 3]
[1, 2, 3]


Note how the second is really a reference to the first so changing one changes the other:

In [65]:
aList[0] = 0
bList[1] = 1
bList.append(4)
print(aList)
print(bList)

[0, 1, 3, 4]
[0, 1, 3, 4]


For a simple list ... we can fix that by simply using list() during our attempt to create the copy:

In [66]:
bList = list(aList)
bList[0] = 999
aList[1] = 998
print(aList)
print(bList)

[0, 998, 3, 4]
[999, 1, 3, 4]


In [67]:
bList.append(19)
print(aList)
print(bList)

[0, 998, 3, 4]
[999, 1, 3, 4, 19]


Mutation is avoided.  Now we can change our two objects independantly.  However, with complex objects like crazyList, this does not work.

The following will illustrate the problem and later, options to get around it are presented.

In [68]:
crazyList2 = list(crazyList)

In [69]:
crazyList2

[array([[1, 2, 3, 4, 5]]),
 array([[[ 0.02764211,  0.26386474,  0.88865395,  0.34781755,  0.72534514],
         [ 0.66574007,  0.73384079,  0.57345696,  0.55389339,  0.87147245],
         [ 0.98128298,  0.88944205,  0.44554892,  0.7894838 ,  0.97115509],
         [ 0.08401461,  0.16542205,  0.00558565,  0.86419018,  0.40254914]],
 
        [[ 0.71687244,  0.72853557,  0.55106175,  0.78393111,  0.69921926],
         [ 0.12197909,  0.93129564,  0.42840254,  0.34314161,  0.12802696],
         [ 0.88812661,  0.75876102,  0.00127025,  0.60469319,  0.28094787],
         [ 0.61414269,  0.69289326,  0.78397186,  0.19069658,  0.76636118]],
 
        [[ 0.28322395,  0.20178467,  0.69215878,  0.80796972,  0.08010599],
         [ 0.47625811,  0.5169806 ,  0.98614882,  0.26718482,  0.4316306 ],
         [ 0.15617731,  0.80081827,  0.37400983,  0.04256393,  0.98771807],
         [ 0.99770397,  0.42965486,  0.2190626 ,  0.24189908,  0.95881218]]]),
 array([[10,  9,  8,  7,  6]]),
 array([[[ 0.0276421

Now we make some changes:

In [25]:
len(crazyList2)-1  # this is the position of the object we want to change

7

In [70]:
crazyList2[7][1] = 13  # this will change element 2 of last object in crazyList2

Now we'll look at just the last object in both "crazyLists" showing what changed:

In [71]:
print(crazyList[7])
print(crazyList2[7])

[10 13  8  7  6]
[10 13  8  7  6]


The "13" replaced the value at this location in both crazyList and crazyList2.  We are not dealing with true copies but rather references to the same data as further illustrated here:

In [30]:
crazyList[7][1] = 9   # change on of them again and both change
print(crazyList[7])
print(crazyList2[7])

[10  9  8  7  6]
[10  9  8  7  6]


So ... how to make a copy that does not mutate?  (we can change one without changing the other)?<br/>
Let's look at some things that don't work first ...

In [72]:
crazyList3 = crazyList[:]  # according to online topics ... this was supposed to work for the reason outlined below
                           # it probably works with some complex objects but does not work with this one
    
# some topics online indicate this should have worked because:
#   * the problem is avoided by "slicing" the original so Python behaves as if the thing you are copying is different
#   * if you used crazyList[2:3] ==> you would get a slice of the original you could store in the copy
#   * [:] utilizes slicing syntax but indicates "give me the whole thing" since by default, empty values are the min and max
#         indexing limits

crazyList3[7][1] = 13      # this will change element 2 of the last object
print(crazyList[7])
print(crazyList3[7])

[10 13  8  7  6]
[10 13  8  7  6]


In [73]:
# what if we do this?  (slice it and then add back a missing element)
crazyList3 = crazyList[:-1]
print(len(crazyList3))
print(len(crazyList))  # crazyList 3 is now one element shorter than crazyList

8
9


In [74]:
crazyList3.append(crazyList[7])  # add back missing element from crazyList
print(len(crazyList3))
print(len(crazyList))

9
9


In [34]:
crazyList3[7][1] = 9        # this will change element 2 of the last object
print(crazyList[7])         # note how again, both lists change
print(crazyList3[7])

[10  9  8  7  6]
[10  9  8  7  6]


Python is hard to fool ... At first, I considered that we might now have two lists, but w/ just element 7 passed in by reference and so it mutates. But this shows our whole lists are still mutating:

In [75]:
print("before:")
print(crazyList[4])
print(crazyList3[4])
crazyList3[4][0] = 14
print("after:")
print(crazyList[4])
print(crazyList3[4])  # try other tests of other elements and you will get same results

before:
[11, 12, 13]
[11, 12, 13]
after:
[14, 12, 13]
[14, 12, 13]


`deepcopy()` comes from the `copy` library and the [commands](https://docs.python.org/2/library/copy.html) are documented at Python.org.  For this situation, this solution seems to work for when mutation is undesirable:

In [76]:
import copy
crazyList4 = copy.deepcopy(crazyList)

In [77]:
print("before:")
print(crazyList[4])
print(crazyList4[4])
crazyList4[4][0] = 15
print("")
print("after:")
print(crazyList[4])
print(crazyList4[4])

before:
[14, 12, 13]
[14, 12, 13]

after:
[14, 12, 13]
[15, 12, 13]


Should even `deepcopy()` not work, this topic online may prove helpful in these situations:  [Stack Overflow: When Deep Copy is not Enough](http://stackoverflow.com/questions/1601269/how-to-make-a-completely-unshared-copy-of-a-complicated-list-deep-copy-is-not).

<a id="indexing" name="indexing"></a>
## Finding The Index of a Value

Suppose we didn't know how to find the element but we knew the value we were looking for?  How to get its index?

In [85]:
print(stupidList)
print(stupidList[1].index(5))  # this works on lists

[[1, 2, 3], [4, 5, 6]]
1


In [88]:
# but for nested lists, you would need to loop through each sublist and handle the error that 
# gets thrown each time it does not find the answer

for element in stupidList:
    try:                  
        test_i = element.index(5)
    except Exception as ex:
        print("%s%s %s" %(type(ex), ":", ex))
        
print(test_i)

<type 'exceptions.ValueError'>: 5 is not in list
1


In [91]:
# this strategy will not work on numpy arrays though
try:
    crazyList[0].index(2)
except Exception as anyE:
    print(type(anyE), anyE)

(<type 'exceptions.AttributeError'>, AttributeError("'numpy.ndarray' object has no attribute 'index'",))


In [93]:
# because we have a list containing numpy arrays, we could look in each one like this:
print(crazyList[0])
np.where(crazyList[0]==2)

[[1 2 3 4 5]]


(array([0], dtype=int64), array([1], dtype=int64))

In [95]:
# the above indicates that 2 lives here:
crazyList[0][0][1]   # started with crazyList[0], then found it at [0][1] inside the data structure

2

In [100]:
# For floating point numbers, the level of precision matters
# details on how this works are presented in this notebook:  TMWP_np_where_and_floatingPoint_numbers.ipynb

# the simple test in the cells that follow should help illustrate the problem and what to do, but 
# see aforementioned notebook for more detail

# to perform a where() test on a structure like this, it is important to note that print()
# rounds the result to 8 decimal places.  The real underlying numbers have more decimal places

print(crazyList2[1]); print("")
print(crazyList2[1][2][3][4])                       # get a number to test with
print("{0:.20}".format(crazyList2[1][2][3][4]))     # show more decimal places of the test number

[[[ 0.02764211  0.26386474  0.88865395  0.34781755  0.72534514]
  [ 0.66574007  0.73384079  0.57345696  0.55389339  0.87147245]
  [ 0.98128298  0.88944205  0.44554892  0.7894838   0.97115509]
  [ 0.08401461  0.16542205  0.00558565  0.86419018  0.40254914]]

 [[ 0.71687244  0.72853557  0.55106175  0.78393111  0.69921926]
  [ 0.12197909  0.93129564  0.42840254  0.34314161  0.12802696]
  [ 0.88812661  0.75876102  0.00127025  0.60469319  0.28094787]
  [ 0.61414269  0.69289326  0.78397186  0.19069658  0.76636118]]

 [[ 0.28322395  0.20178467  0.69215878  0.80796972  0.08010599]
  [ 0.47625811  0.5169806   0.98614882  0.26718482  0.4316306 ]
  [ 0.15617731  0.80081827  0.37400983  0.04256393  0.98771807]
  [ 0.99770397  0.42965486  0.2190626   0.24189908  0.95881218]]]

0.958812178544
0.95881217854380618171


In [101]:
# Warning! If you re-run this notebook, new random nubers are generated and the value used for the test in this
#          cell will probably then fail.  To fix this, re-run previous cell and copy in the final number shown
#          above up to at least 17 decimal places.

print(np.where(crazyList2[1]==0.95881217854380618)) # number copied from output of previous line up to 17 decimal places
                                                    # np.where() can find this, but will also return other values
                                                    # that match up to the first 16 decimal places (if they exist)
                                                    # precision appears to be up to 16 decimal places on a 32 bit machine

(array([2], dtype=int64), array([3], dtype=int64), array([4], dtype=int64))


In [118]:
# np.isclose
# for finding less precise answers:  finds numbers that "are close"

print(np.isclose(crazyList2[1], 0.95881))
print("")
print(np.where(np.isclose(crazyList2[1], 0.95881)))  # note that when numbers are "close" this returns multiple values
                                                     # in this case (crazyList2) only one number was "close"
                                                     # more detailed testing is provided in: 
                                                     #    TMWP_np_where_and_floatingPoint_numbers.ipynb

[[[False False False False False]
  [False False False False False]
  [False False False False False]
  [False False False False False]]

 [[False False False False False]
  [False False False False False]
  [False False False False False]
  [False False False False False]]

 [[False False False False False]
  [False False False False False]
  [False False False False False]
  [False False False False  True]]]

(array([2], dtype=int64), array([3], dtype=int64), array([4], dtype=int64))


Related help topics for additional research and reading:
 - [Finding the Index - some options on Stack Overflow](http://stackoverflow.com/questions/176918/finding-the-index-of-an-item-given-a-list-containing-it-in-python)
 - [numpy.where()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html)
 - [numpy.isclose()](https://docs.scipy.org/doc/numpy-1.10.4/reference/generated/numpy.isclose.html)
 - [Pandas Dataframe Indexing Tutorial](https://www.novixys.com/blog/pandas-tutorial-select-dataframe/)

The End ...